In [1]:
import acquire as a
import prepare as p
import explore as e
import model as m

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
train, validate, test = p.split_data((p.prepare_data(a.acquire_data())),'churn')

File exists - reading CSV file


In [3]:
len(train.columns)

21

In [4]:
cat_cols = train.columns[train.nunique() < 10].drop('churn').tolist()
num_cols = train.columns[train.nunique() >= 10].drop('customer_id').tolist()

In [34]:
for col in num_cols:
    print(f'{col} vs churn:')
    churned = train[train.churn == 'Yes'][col]
    active = train[train.churn == 'No'][col]
    
    t, p = stats.shapiro(churned)
    if p < α:
        print('The distributions are not normally distributed.')
    else: 
        print('The distributions are normal.')
        
    t, p = stats.shapiro(active)
    if p < α:
        print('The distributions are not normally distributed.')
    else: 
        print('The distributions are normal.')
    

    α = .05
    t, p = stats.levene(churned, active)
    if p < α:
        print(f'There is an inequality of variance in our two samples.')
    else:
        print(f'There is equal variance in our two samples.')
    print(churned.shape, active.shape, train.shape)
    print()
    
    
        

# Ha: The distributions are not normally distributed

tenure vs churn:
The distributions are not normally distributed.
The distributions are not normally distributed.
There is an inequality of variance in our two samples.
(1121,) (3104,) (4225, 21)

monthly_charges vs churn:
The distributions are not normally distributed.
The distributions are not normally distributed.
There is an inequality of variance in our two samples.
(1121,) (3104,) (4225, 21)

total_charges vs churn:
The distributions are not normally distributed.
The distributions are not normally distributed.
There is an inequality of variance in our two samples.
(1121,) (3104,) (4225, 21)



In [9]:
cat_cols_w_rel = []
cat_cols_wo_rel = []

for col in cat_cols:
    observed = pd.crosstab(train[col],train.churn)
    α = .05
    chi2, p, dof, expected = stats.chi2_contingency(observed)
    
    if p < α:
        cat_cols_w_rel.append(col)
    else:
        cat_cols_wo_rel.append(col)

In [10]:
churned = train[train.churn == 'Yes'][col]
active = train[train.churn == 'No'][col]
quant_cols_w_rel = []
quant_cols_wo_rel = []

for col in num_cols:
    t,p = stats.mannwhitneyu(churned[col], active[col])
    α = .05
    
    if p < α:
        quant_cols_w_rel.append(col)
    else:
        quant_cols_wo_rel.append(col)

In [7]:
for col in cat_cols_w_rel:
    print(pd.crosstab(train[col],train.churn,normalize=0))
    print()

churn                 No       Yes
senior_citizen                    
No              0.765758  0.234242
Yes             0.578947  0.421053

churn          No       Yes
partner                    
No       0.669820  0.330180
Yes      0.806484  0.193516

churn             No       Yes
dependents                    
No          0.687077  0.312923
Yes         0.845548  0.154452

churn                   No       Yes
multiple_lines                      
No                0.757185  0.242815
No phone service  0.735714  0.264286
Yes               0.709010  0.290990

churn                      No       Yes
online_security                        
No                   0.585262  0.414738
No internet service  0.926407  0.073593
Yes                  0.852196  0.147804

churn                      No       Yes
online_backup                          
No                   0.603854  0.396146
No internet service  0.926407  0.073593
Yes                  0.781577  0.218423

churn                      No    

In [ ]:
sns.countplot(train)